This file contains some additional baselines used in the paper, namely:
1. PI-EW (plugin)
2. FW val 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import csv
import numpy as np
import tensorflow as tf
from sklearn import metrics 

BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")

import utils.metrics
import utils.record

Using TensorFlow backend.


In [3]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
RANDOM_SEED = CONFIG['random_seed']
EPOCHS = CONFIG["experiment_configs"][DATASET]["epochs"]
BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]
IMAGE_X_SIZE = CONFIG["experiment_configs"][DATASET]["image_x_size"]
IMAGE_Y_SIZE = CONFIG["experiment_configs"][DATASET]["image_y_size"]
IMAGE_SIZE = (IMAGE_Y_SIZE, IMAGE_X_SIZE)
HYPER_VAL_SPLIT = CONFIG['experiment_configs'][DATASET]['hyper_val_split']

print(DATASET, RANDOM_SEED)

# folders for processed, models
DATA_F = os.path.join(BASE_DIR, f"data/{DATASET}/")
PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/{DATASET}/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/{DATASET}/rs={RANDOM_SEED}')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')

BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="base") # mt = model_type

# base model saved here
if not os.path.exists(BASE_MODEL_SAVEPATH):
    print(f"warning: no model has been run for rs={RANDOM_SEED}")

adience 55


In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "val"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

hyper_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "hyper_val"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "test"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

Found 199 files belonging to 2 classes.
Found 200 files belonging to 2 classes.
Found 3585 files belonging to 2 classes.


In [5]:
'''
This will standardize the pixel data
'''
def preprocess(imgs, labels):
    # turn from <0..255> to <0..1>
    imgs = imgs / 255.0
    means = np.array( [0.5, 0.5, 0.5] )
    stds = np.array( [0.5, 0.5, 0.5] )
    imgs = (imgs - means) / stds
    return imgs, labels

In [6]:
val_ds = val_ds.map(preprocess)
hyper_val_ds = hyper_val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

In [7]:
model = utils.utils.make_resnet(
    depth=2,
    random_state=RANDOM_SEED,
    input_shape=(*IMAGE_SIZE, 3),
    nc=2,
)
model.load_weights(BASE_MODEL_SAVEPATH)

In [8]:
preds_val, y_val = utils.utils.compute_preds(model, val_ds)
preds_hyper_val, y_hyper_val = utils.utils.compute_preds(model, hyper_val_ds)
preds_test, y_test = utils.utils.compute_preds(model, test_ds)

100%|██████████| 113/113 [00:23<00:00,  4.75it/s]


In [34]:
preds_val_full = np.concatenate([preds_val, preds_hyper_val])
y_val_full = np.concatenate([y_val, y_hyper_val])

In [41]:
METRIC = 'F-measure'
CLASSES = 2

# for FW_val
NUM_ITERS = 100

In [49]:
pred_eta_model = {
    "val_full": preds_val_full,
    "val": preds_val,
    "hyper_val": preds_hyper_val,
    "test": preds_test
}

In [14]:
from utils.papers_piew import plugin

In [61]:
mval_val, mval_test, final_weights = plugin(METRIC, y_val_full, y_test, pred_eta_model, CLASSES)

In [62]:
os.makedirs(RESULTS_DIR, exist_ok=True)
savepath = os.path.join(RESULTS_DIR, f"results_{DATASET}.csv")
saver = utils.record.Results_Recorder(savepath, DATASET)
saver.save(RANDOM_SEED, METRIC, "plugin", mval_val, mval_val, mval_test, None)
saver.close()

Results file exists, appending to it...


In [40]:
from utils.papers_fw_val import FW_val

In [51]:
mval_val_list, mval_hyper_val_list, mval_test_list, grad_norm_list = FW_val(
    NUM_ITERS,
    METRIC,
    y_val,
    y_hyper_val,
    y_test, 
    pred_eta_model,
    CLASSES,
    RANDOM_SEED
)

In [52]:
best_idx = np.argmax(mval_hyper_val_list)
mval_val = mval_val_list[best_idx]
mval_hyper_val = mval_hyper_val_list[best_idx]
mval_test = mval_test_list[best_idx]

In [55]:
mval_val, mval_hyper_val, mval_test

(0.7734674200404332, 0.7246409793543266, 0.7388613058579313)

In [57]:
os.makedirs(RESULTS_DIR, exist_ok=True)
savepath = os.path.join(RESULTS_DIR, f"results_{DATASET}.csv")
saver = utils.record.Results_Recorder(savepath, DATASET)
saver.save(RANDOM_SEED, METRIC, "fw_val", mval_val, mval_hyper_val, mval_test, None)
saver.close()

Results file exists, appending to it...
